In [1]:
import random
from re import match
import pysam
import numpy as np
import pandas as pd
import os
import argparse
import json
from copy import deepcopy

from convert import convert_hap_samples_to_dataframe

In [24]:
def get_file(data_path):
    '''从文件夹中，得到各文件的路径'''
    for filename in os.listdir(data_path):
        if filename.endswith('.gz'):
            hap_file = os.path.join(data_path,filename)
        if filename.endswith('.sample'):
            samples_file = os.path.join(data_path,filename)
        if filename.endswith('.vcf'):
            vcf_file = os.path.join(data_path,filename)
        if filename.endswith('.ped'):
            ped_file = os.path.join(data_path,filename)
    assert (hap_file and samples_file and vcf_file and ped_file),'missing data file'
    return  hap_file,samples_file,vcf_file,ped_file

def parse_args():
    parser = argparse.ArgumentParser(description="Simulate generations of families and populations")
    parser.add_argument("dataset", help="Input data file with initial individuals")
    parser.add_argument("output_file_path", help="path for saving simulation files ")
    parser.add_argument("-g", "--num_generations", type=int, default=3, help="Number of generations to simulate (default: 3)")
    parser.add_argument("-c", "--num_couples", type=int, default=10, help="Number of initial couples (default: 10)")
    parser.add_argument("-l", "--num_linkages", type=int, default=1000, help="Number of consecutive non-recombining positions (default: 1000)")
    parser.add_argument("-p", "--num_keys", type=int, default=0, help="key points for linkage interval (default: 0)")
    parser.add_argument("-k", "--num_kids", type=int, default=2, help="Number of child for every couple (default: 2)")
    parser.add_argument("-r", "--num_recombination", type=int, default=5, help="Number of intervals for recombination (default: 5)")

    return parser.parse_args()

def choice_person(df_data,num_couples,start,end):
    res0 = pd.DataFrame()
    select_people = {}
    num_people = df_data.shape[0]//2
    person_list = df_data.index.tolist()
    df_data.columns = np.arange(df_data.shape[1])
    for i in range(2*num_couples):
        index = random.randint(0,num_people)
        select_people[i] = person_list[2*index]
        select_data = deepcopy(df_data.iloc[index*2:index*2+2])
        select_data['personID'] = [i,i]
        res0 = pd.concat([res0,select_data])
    res0.set_index(np.arange(4*num_couples),drop=True,inplace=True)
    for i in range(res0.shape[0]):
        res0.iloc[i,start:end] = res0.iloc[i,start]
    return res0,select_people

def generate_linkage_intervals(num_linkage,num_locus,num_keys=0):
    linkage_init = random.randint(0,num_locus-num_linkage)
    keys = np.random.choice(np.arange(1,num_linkage-1), num_keys,replace=False) + linkage_init
    linkage_keys = np.sort(np.append(keys,[linkage_init,linkage_init+num_linkage+1]))
    return linkage_keys

def generate_recombination_intervals(num_recombinations,num_locus,linkage_keys):
    start = linkage_keys[0]
    end = linkage_keys[-1]
    num_linkage = end-start
    interval1 = range(0,start)
    interval2 = range(end+1,num_locus+1)
    points1 = start*num_recombinations//(num_locus-num_linkage)
    points2 = num_recombinations - points1
    recombination_points1 = sorted(random.sample(interval1, 2 * points1)) 
    recombination_points2 = sorted(random.sample(interval2, 2 * points2))
    recombination_points = recombination_points1 + recombination_points2
    recombination_intervals = [(recombination_points[i], recombination_points[i + 1]) for i in range(0, len(recombination_points), 2)]
    return recombination_intervals

def is_variant_in_recombination_intervals(variant_pos, recombination_intervals):
    '''检查一个变异位点是否在给定的重组区间内'''
    if recombination_intervals!=[]:
        for start, end in recombination_intervals:
            if start <= variant_pos <= end:
                return True
    return False

def sim_meosis(res,num_recombinations,linkage_keys):
    num_locus = res.shape[1]-1
    start = linkage_keys[0]
    end = linkage_keys[-1]
    temp = deepcopy(res[res.index%2==0])
    for i in range(temp.shape[0]):
        if num_recombinations>0:
            recombination_intervals = generate_recombination_intervals(num_recombinations,num_locus,linkage_keys)
        else:
            recombination_intervals = []
        for j in range(num_locus):
            if j<start or j>=end:
                if is_variant_in_recombination_intervals(j,recombination_intervals):
                    temp.iloc[i,j] = res.iloc[i+1,j]
            # else:
            #     # index = np.searchsorted(linkage_keys,j,side='right') - 1   # type: ignore
            #     temp.iloc[i,j] = res.iloc[i,start]

    return temp

def couple_legal(person1,person2,family=None):
    if not family:
        return True
    else:
        if person1 in family.keys() and person2 in family.keys():
            if family[person1] == family[person2]:
                return False
            else:
                return True
        else:
            return True

def select_parent(person_rest,family):
    persons = deepcopy(person_rest)
    parent1,parent2 = random.sample(persons,2)
    while not couple_legal(parent1,parent2,family):
        parent1,parent2 = random.sample(persons,2)
    persons.remove(parent1)
    persons.remove(parent2)
    return persons,parent1,parent2


In [3]:
num_generation = 1
num_couple = 5
num_child = 2
num_linkage = 1000
num_recombination = 0
num_keys = 0

In [4]:
data_path = "/home/anran/paternity/family-data/sim_seg1" 
hap_file,samples_file,_,_=get_file(data_path)
df_data = convert_hap_samples_to_dataframe(hap_file,samples_file)
df_data.drop(['ID','REF','ALT'],axis=0,inplace=True)
num_locus = df_data.shape[1]
print(num_locus)

74560


In [5]:
linkage_keys = generate_linkage_intervals(num_linkage,num_locus,num_keys)
start = linkage_keys[0]
end = linkage_keys[-1]
start,end

(49310, 50311)

In [25]:
res0,family = choice_person(df_data,num_couple,start,end)

In [26]:
res0.iloc[:,start:start+5]

,49310,49311,49312,49313,49314
0,0,0,0,0,0
1,1,1,1,1,1
2,1,1,1,1,1
3,0,0,0,0,0
4,1,1,1,1,1
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,1,1,1,1,1
9,0,0,0,0,0


In [17]:
for i in range(3):
    df.iloc[i,0:2] = df.iloc[i,0]
df

,0,1,2
0,0,0,2
1,3,3,5
2,6,6,8


In [31]:
a = np.array([1,2])
type(list(a))

numpy.int64

# 算法ipad

In [212]:
from copy import deepcopy
from os import replace

def choice_person(df_data,num_couples):
    res0 = pd.DataFrame()
    select_people = {}
    num_people = df_data.shape[0]//2
    person_list = df_data.index.tolist()
    df_data.columns = np.arange(df_data.shape[1])
    for i in range(2*num_couples):
        index = random.randint(0,num_people)
        select_people[i] = person_list[2*index]
        select_data = deepcopy(df_data.iloc[index*2:index*2+2])
        select_data['personID'] = [i,i]
        res0 = pd.concat([res0,select_data])
    res0.set_index(np.arange(4*num_couples),drop=True,inplace=True)
    return res0,select_people

In [165]:
def generate_linkage_intervals(num_linkage,num_locus):
    '''生成给定数量的重组间隔，即随机选择一定数量的变异位点作为重组点，形成相邻两个变异位点之间的重组区间。'''
    linkage_init = random.randint(0,num_locus-num_linkage)
    linkage_interval = np.arange(num_linkage)+linkage_init
    return linkage_interval

def generate_recombination_intervals(num_recombinations,num_locus,linkage_interval):
    num_linkage = len(linkage_interval)
    interval1 = range(0,linkage_interval[0])
    interval2 = range(linkage_interval[-1]+1,num_locus+1)
    points1 = linkage_interval[0]*num_recombinations//(num_locus-num_linkage)
    points2 = num_recombinations - points1
    recombination_points1 = sorted(random.sample(interval1, 2 * points1)) 
    recombination_points2 = sorted(random.sample(interval2, 2 * points2))
    recombination_points = recombination_points1 + recombination_points2
    recombination_intervals = [(recombination_points[i], recombination_points[i + 1]) for i in range(0, len(recombination_points), 2)]
    return recombination_intervals

def is_variant_in_recombination_intervals(variant_pos, recombination_intervals):
    '''检查一个变异位点是否在给定的重组区间内'''
    for start, end in recombination_intervals:
        if start <= variant_pos <= end:
            return True
    return False

In [166]:
def sim_meosis(res,num_recombinations,linkage_interval):
    num_locus = res.shape[1]-1
    temp = deepcopy(res[res.index%2==0])
    for i in range(temp.shape[0]):
        recombination_intervals = generate_recombination_intervals(num_recombinations,num_locus,linkage_interval)
        for j in range(num_locus):
            if is_variant_in_recombination_intervals(j,recombination_intervals):
                temp.iloc[i,j] = res.iloc[i+1,j]
    return temp

In [122]:
def couple_legal(person1,person2,family=None):
    if not family:
        return True
    else:
        if person1 in family.keys() and person2 in family.keys():
            if family[person1] == family[person2]:
                return False
            else:
                return True
        else:
            return True

def select_parent(person_rest,family):
    persons = deepcopy(person_rest)
    parent1,parent2 = random.sample(persons,2)
    while not couple_legal(parent1,parent2,family):
        parent1,parent2 = random.sample(persons,2)
    persons.remove(parent1)
    persons.remove(parent2)
    return persons,parent1,parent2

In [148]:
def some():
    # args = parse_args()

    num_generation = 2
    num_couple = 3
    num_child = 2
    family = {}
    result_df = pd.DataFrame()
    num_linkage = 100
    num_recombination = 5

    data_path = "/home/anran/paternity/family-data/sim_seg1" 
    hap_file,samples_file,_,_=get_file(data_path)
    df_data = convert_hap_samples_to_dataframe(hap_file,samples_file)
    df_data.drop(['ID','REF','ALT'],axis=0,inplace=True)
    num_locus = df_data.shape[1]
    
    linkage_interval = generate_linkage_intervals(num_linkage,num_locus)
    # 初代的人数据
    res0,select_people = choice_person(df_data,num_couples)
    print(select_people)
    result_df = pd.concat([result_df,res0])
    personID = 2*num_couples
    # print(res0.shape,result_df.shape,personID)

    for g in range(num_generation):
        print('#####################',g)
        for k in range(num_child):
            print('#########',k)
            temp = sim_meosis(res0,num_recombination,linkage_interval)
            persons_rest = list(temp['personID'].values)
            # print(temp.shape,persons_rest)
            while persons_rest!=[]:
                persons_rest,parent1,parent2 = select_parent(persons_rest,family)
                # print('-----------------',parent1,parent2,persons_rest)
                family[personID] = [parent1,parent2]
                res_temp = deepcopy(temp[temp['personID'].isin([parent1,parent2])])
                res_temp['personID'] = [personID,personID]
                result_df = pd.concat([result_df,res_temp])
                print(result_df.shape)
                personID += 1
        # display(result_df)
        res0 = result_df.iloc[2*personID-4*num_couple:2*personID-2]
        # display(res0)
    return result_df,res0


In [194]:
num_generation = 5
num_couple = 10
num_child = 2
# family = {}
result_df = pd.DataFrame()
num_linkage = 1000
num_recombination = 6

In [193]:
data_path = "/home/anran/paternity/family-data/sim_seg1" 
hap_file,samples_file,_,_=get_file(data_path)
df_data = convert_hap_samples_to_dataframe(hap_file,samples_file)
df_data.drop(['ID','REF','ALT'],axis=0,inplace=True)
num_locus = df_data.shape[1]

In [221]:
linkage_interval = generate_linkage_intervals(num_linkage,num_locus)
# 初代的人数据
res0,family = choice_person(df_data,num_couple)
print(family)
result_df = pd.concat([result_df,res0])
personID = 2*num_couple

{0: 'HG03120_0', 1: 'NA21118_0', 2: 'HG01538_0', 3: 'HG02104_0', 4: 'HG01255_0', 5: 'HG03577_0', 6: 'HG03196_0', 7: 'HG00446_0', 8: 'HG02604_0', 9: 'HG01098_0', 10: 'HG00188_0', 11: 'HG02838_0', 12: 'HG01488_0', 13: 'HG03499_0', 14: 'HG00582_0', 15: 'HG04160_0', 16: 'HG00383_0', 17: 'HG02972_0', 18: 'HG02892_0', 19: 'HG03899_0'}


In [222]:
for g in range(num_generation):
    print('#####################',g)
    couple = []
    for k in range(num_child):
        print('#########',k)
        temp = sim_meosis(res0,num_recombination,linkage_interval)
        persons_rest = list(map(int,list(temp['personID'].values)))
        # print(temp.shape,persons_rest)
        if k==0:
            while persons_rest!=[]:
                persons_rest,parent1,parent2 = select_parent(persons_rest,family)
                print('-----------------',parent1,parent2,persons_rest)
                couple.append((parent1,parent2))
                family[personID] = [parent1,parent2]
                res_temp = deepcopy(temp[temp['personID'].isin([parent1,parent2])])
                res_temp['personID'] = [personID,personID]
                result_df = pd.concat([result_df,res_temp])
                personID += 1
        else:
            for (parent1,parent2) in couple:
                family[personID] = [parent1,parent2]
                res_temp = deepcopy(temp[temp['personID'].isin([parent1,parent2])])
                res_temp['personID'] = [personID,personID]
                result_df = pd.concat([result_df,res_temp])
                personID += 1
    #     display(result_df)
    res0 = result_df.iloc[2*personID-4*num_couple:2*personID]
    res0.set_index(np.arange(res0.shape[0]),drop=True,inplace=True)

##################### 0
######### 0
----------------- 4 1 [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
----------------- 0 17 [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19]
----------------- 14 10 [2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 18, 19]
----------------- 9 19 [2, 3, 5, 6, 7, 8, 11, 12, 13, 15, 16, 18]
----------------- 15 6 [2, 3, 5, 7, 8, 11, 12, 13, 16, 18]
----------------- 11 8 [2, 3, 5, 7, 12, 13, 16, 18]
----------------- 5 16 [2, 3, 7, 12, 13, 18]
----------------- 13 3 [2, 7, 12, 18]
----------------- 18 7 [2, 12]
----------------- 12 2 []
######### 1
##################### 1
######### 0
----------------- 33 29 [20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 34, 35, 36, 37, 38, 39]
----------------- 34 21 [20, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 35, 36, 37, 38, 39]
----------------- 22 25 [20, 23, 24, 26, 27, 28, 30, 31, 32, 35, 36, 37, 38, 39]
----------------- 36 20 [23, 24, 26, 27, 28, 30, 31, 32, 35, 37, 38, 39]
-----------------

In [223]:
family['start'] = int(linkage_interval[0])
family['end'] = int(linkage_interval[-1])

In [224]:
result_df.to_csv('/home/anran/paternity/family-data/sim_data/data2.csv')
# flist=[family]
with open("/home/anran/paternity/family-data/sim_data/info2.json","w",encoding='utf-8') as f: ## 设置'utf-8'编码
    # f.write(json.dumps(family, ensure_ascii=False) + '\n') 
    json.dump(family,f)

- py文件验证

In [1]:
import numpy as np
import pandas as pd
import pysam
import random
from convert import convert_hap_samples_to_dataframe
import os
import math
from copy import deepcopy
import numba
import os
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
import subprocess
import json

import warnings
warnings.filterwarnings('ignore')

K=10


def get_file(data_path):
    '''从文件夹中，得到各文件的路径'''
    for filename in os.listdir(data_path):
        if filename.endswith('.gz'):
            hap_file = os.path.join(data_path,filename)
        if filename.endswith('.sample'):
            samples_file = os.path.join(data_path,filename)
        if filename.endswith('.vcf'):
            vcf_file = os.path.join(data_path,filename)
        if filename.endswith('.ped'):
            ped_file = os.path.join(data_path,filename)
    assert (hap_file and samples_file and vcf_file and ped_file),'missing data file'
    return  hap_file,samples_file,vcf_file,ped_file

'''
# def select_locus(vcf, n, father, mother, child):
#     # 从vcf中随机抽出父母和孩子n行的数据
#     df = pd.DataFrame(columns=['locus','moGT','faGT', 'chGT','f0'])
#     for record in vcf:   # 先将所有数据读进df
#         locus = (record.chrom,str(record.pos))
#         f0 = 1-record.info['AF'][0]
#         if f0<1:  # 除去f0=1的情况，不然会导致之后 arctanh(1)=inf
#             mo = record.samples[mother]['GT']
#             ch = record.samples[child]['GT']
#             if (mo!=(0,1) and mo!=(1,0)) or (ch!=(0,1) and ch!=(1,0)):
#                 fa = record.samples[father]['GT']
#                 df.loc[len(df.index)] = [locus,mo,fa,ch,f0] # type: ignore
#     num_row = df.shape[0]
#     assert num_row>=n,'not enough locus to select'
#     df_select = df.sample(n,replace=False,axis=0)
#     df_select.set_index('locus',drop=True,inplace=True)
#     return df_select

# def select_locus(vcf, n, father, mother, child):
#     # 从vcf中随机抽出<<连续的父母>>和孩子n行的数据
#     df = pd.DataFrame(columns=['locus','moGT','faGT', 'chGT','f0'])
#     for record in vcf:   # 先将所有数据读进df
#         locus = (record.chrom,str(record.pos))
#         f0 = 1-record.info['AF'][0]
#         if f0<1:  # 除去f0=1的情况，不然会导致之后 arctanh(1)=inf
#             mo = record.samples[mother]['GT']
#             ch = record.samples[child]['GT']
#             if (mo!=(0,1) and mo!=(1,0)) or (ch!=(0,1) and ch!=(1,0)):
#                 fa = record.samples[father]['GT']
#                 df.loc[len(df.index)] = [locus,mo,fa,ch,f0] # type: ignore
#     num_row = df.shape[0]
#     assert num_row>=n,'not enough locus to select'
#     i = random.randint(n, num_row)
#     df_select = df.iloc[i-n:i,:]
#     df_select.set_index('locus',drop=True,inplace=True)
#     return df_select
'''

def calcul_XY(df_select):
    '''遍历选择的位点计算X,Y和c, 返回logX,logY and c'''
    log_mu = -8 # 默认突变概率为1e-8
    log2 = np.log10(2)

    n = df_select.shape[0]
    log_x = 0
    log_y = 0
    c = []
    
    for i in range(n):
        father = df_select['faGT'][i]
        mother = df_select['moGT'][i]
        child = df_select['chGT'][i]
        if child == (0,0):
            if mother == (0,1) or mother == (1,0):
                log_y = log_y - log2
                c.append(0)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - 2*log2
                elif father == (0,0):
                    log_x = log_x - log2
                else:
                    log_x = log_x + log_mu - log2

            elif mother == (0,0):
                c.append(0)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - log2
                elif father == (0,0):
                    pass
                else:
                    log_x = log_x + log_mu

            else:
                log_y = log_y + log_mu
                c.append(0)
                if father == (0,1) or father == (1,0):
                    log_x = log_x + log_mu -log2
                elif father == (0,0):
                    log_x = log_x + log_mu
                else:
                    log_x = log_x + 2*log_mu

        elif child == (0,1) or child == (1,0):
            if mother == (0,1) or mother == (1,0):
                c.append(0)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - log2
                elif father == (0,0):
                    log_x = log_x - log2
                else:
                    log_x = log_x - log2
            #    assert 1==0,'mother and child are both heteGT'

            elif mother == (0,0):
                c.append(1)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - log2
                elif father == (0,0):
                    log_x = log_x + log_mu
                else:
                    pass

            else:
                c.append(0)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - log2
                elif father == (0,0):
                    pass
                else:
                    log_x = log_x + log_mu

        else:
            if mother == (0,1) or mother == (1,0):
                log_y = log_y - log2
                c.append(1)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - 2*log2
                elif father == (0,0):
                    log_x = log_x + log_mu - log2
                else:
                    log_x = log_x -log2

            elif mother == (0,0):
                log_y = log_y + log_mu
                c.append(1)
                if father == (0,1) or father == (1,0):
                    log_x = log_x + log_mu - log2
                elif father == (0,0):
                    log_x = log_x + 2* log_mu
                else:
                    log_x = log_x + log_mu

            else:
                c.append(1)
                if father == (0,1) or father == (1,0):
                    log_x = log_x - log2
                elif father == (0,0):
                    log_x = log_x + log_mu
                else:
                    pass
    c = np.array(c)
    # print('X,Y,len_c:',log_x,log_y,c.shape)
    return log_x,log_y,c


def calcul_pi_ind(df_select,log_x,log_y,c):
    '''计算独立情况的亲权系数'''
    f0 = df_select['f0'].values
    pr_ind = (np.log10(np.where(c==0,f0,1-f0))).sum() # type: ignore
    return log_x - log_y - pr_ind

@numba.jit(nopython=True)
def freq2(x,y):
    '''返回bool列表,True表示x,y对应位置同时为0'''
    return (x==0)&(y==0)

def calcul_fij(df_data,fi,locus_list,n):
    num_row = df_data.shape[0]
    fij = np.zeros((n,n))
    for i in range(n):  # 因为fij对称，只遍历上三角矩阵
        col1 = df_data[locus_list[i]].to_numpy().astype(int)   # 这里类型的变换，是为了使用numba加速
        for j in range(i+1,n):
            col2 = df_data[locus_list[j]].to_numpy().astype(int)
            fij[i,j] = freq2(col1,col2).sum()/num_row  # 统计两列数据同时为０的概率
            fij[j,i] = fij[i,j]
    for i in range(n):
        fij[i,i] = fi[i]
    return fij

def freq_pc(df_data,locus_list,fi,n,pc):
    fij = calcul_fij(df_data,fi,locus_list,n)
    return (1-pc)*fi + pc/2,(1-pc)*fij + pc/4

def cMat(fi,fij,n,alpha):
    cmat = fij - fi*(fi.reshape(n,1))
    print('\t c的最小特征值',np.min(np.linalg.eigvals(cmat)))
    cmat = np.where(cmat>alpha,cmat,0)
    return cmat

def inv_eig(cmat):
    D,V = np.linalg.eig(cmat)
    lambda_inv= np.diag(list(map(lambda x: x.real/(0.001+x.real**2), D)))
    inv_C2 = np.dot(np.dot(V, lambda_inv), V.T).real
    return inv_C2

def invc_glassoR(cmat,rho1,rho2):
    c_path = '/home/anran/paternity/version4/cmat.txt'
    invc_path = '/home/anran/paternity/version4/invc.txt'
    invg_path = "/home/anran/paternity/version4/invc_glasso.csv"
    np.savetxt(c_path,cmat)
    np.savetxt(invc_path,inv_eig(cmat))
    r_path = f"/home/anran/paternity/version4/invc.R"
    r_command = ['Rscript',r_path]
    r_command.extend(map(str,[c_path, invc_path, invg_path,rho1, rho2]))
    subprocess.run(r_command,check=True)
    invc = pd.read_csv(invg_path,sep=' ').values
    if(os.path.isfile(c_path)):  # 删除保存的数据文件
        os.remove(c_path)
        os.remove(invc_path)
        os.remove(invg_path)
    return invc


def calcul_e(invc,fi,n):
    res = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            a = 2*fi[i]*fi[j]
            delta = 1-4*a*invc[i, j]
            if delta<0 or fi[i]==0 or fi[j]==0:
                res[i,j] = -invc[i,j]
            elif delta==0:
                res[i,j] =  -1/(2*a)
            else:
                s1 = (-1+math.sqrt(delta))/(2*a)
                s2 = (-1+math.sqrt(delta))/(2*a)
                if abs(s1-invc[i,j])>abs(s2-invc[i,j]):  # 当有2实根，选择距-invc较近的根
                    res[i,j] = s2
                else:
                    res[i,j] = s1
    print('\t emat:',res.min(),res.max())
    return res


def margin(fi,emat,n):
    e = deepcopy(emat)
    for i in range(n):  # h计算时，对j!=i的eij项操作
        e[i,i] = 0
    h = np.arctanh(fi) + (e**2 * fi.reshape(n,1) * (1-fi**2)).sum(axis=1) - (e*fi).sum(axis=1)
    print('\t h:',h.min(),h.max())
    return h


def get_b(k):
    '''根据k生成b, 维度为(2^k,k), 一行是一个样本'''
    num_rows = 2**k  # 矩阵b的行数
    b = np.zeros((num_rows, k), dtype=int)
    for i in range(num_rows):
        binary_representation = format(i, '0' + str(k) + 'b')  # 将整数i转换为k位的二进制字符串
        for j in range(k):
            b[i, j] = int(binary_representation[j])
    return b

def select_emat(seq,j,emat,k):
    '''对于第j个位点, 只考虑它及其前面的k-1个位点
    取出emat中的子方阵[j-k+1.j-k+1]->[j,j]
    根据seq计算对应的系数(1 or -1)'''
    s = np.tile(seq,(k,1))
    mask = np.logical_not(s ^ s.T)  # 00/11->1,01/10->0
    mask = np.where(mask==0,-1,mask)  
    return np.triu(emat[j-k+1:j+1,j-k+1:j+1]*mask,1)

def get_addition(j,k,b,e,h):
    '''返回E向量, 维度(2**k,)'''
    coef = np.zeros(2**k)
    for i in range(2**k):
        seq = b[i,:]
        mask = np.where(np.logical_not(seq[-1]^seq[:-1]),1,-1)
        coef[i] = (e[j,j-k+1:j] * mask).sum() + h[j]*(-1)**seq[-1]
    return coef

def calcul_Z(e,h,k,n):
    z = np.zeros((2**k,n-k+1))
    b = get_b(k)

    for i in range(2**k):  # 计算ｚ的第一列数据z[0]
        seq = b[i,:]
        z[i,0] = (h[:k]*np.where(seq==0,1,-1)).sum()+select_emat(seq,k-1,e,k).sum()
    
    for l in range(1,n-k+1): # 循环计算z[1]到z[n-k], l指向z的index, 对应e/h的index为 l+(k-1)
        coef = get_addition(l+k-1,k,b,e,h)
        for i in range(2**k):
            seq = b[i,:]
            l0 = (i-seq[-1]) // 2  # 二进制数右移移位，最高位为0
            l1 = (l0 + 2**(k-1)) # 二进制数右移移位，最高位为１
            z0 = min(z[l0,l-1],z[l1,l-1])
            z1 = max(z[l0,l-1],z[l1,l-1])
            if z1-z0>200:
                z[i,l] = coef[i] + z1
            else: 
                z[i,l] = coef[i] + z0 + np.log(np.exp(1)+np.exp(z1-z0))
    z_min = z[:,-1].min()
    log10_z = np.log10(np.exp(z[:,-1]-z_min).sum()) + np.log10(np.exp(1))*z_min
    return log10_z

def calcul_num(c, e, h, n, k):
    '''输入e维度N*N
    返回10的指数部分'''
    nums = np.zeros(n-k+1)
    sum_h = (h*np.where(c==0,1,-1)).sum()

    nums[0] = select_emat(c[:k],k-1,e,k).sum()

    for j in range(k,n):
        l = j-k+1  # nums中的index
        mask = np.where(np.logical_not(c[j]^c[l:j]),1,-1)  # 使用c_j与其前k-1个位点计算mask
        coef = (e[j,l:j] * mask).sum()
        nums[l] = coef + nums[l-1]
    return (nums[-1]+sum_h)*np.log10(np.exp(1))


# def get_fi(vcf_file,n):
#     '''select N locus from vcf_file'''
#     vcf = pysam.VariantFile(vcf_file)
#     f0=[]
#     for record in vcf:
#         f1 = record.info['AF'][0]
#         locus = (record.chrom,str(record.pos))
#         if f1<0.8 and f1>0.2:
#             f0.append((locus,1-f1))
#     l = random.randint(0,len(f0)-n)
#     freq0 = dict(f0[l:l+n])
#     fi = pd.Series(freq0)
#     return fi

def get_fi(df_data,start,end):
    f0 = {}
    totle = df_data.shape[0]
    columns = df_data.columns.to_list()
    print('linkage aera: ',columns[start],' to ', columns[end])
    for i in range(start,end+1):
        locus = columns[i]
        freq = (df_data[locus]==0).sum()/totle
        f0[locus] = freq
    fi = pd.Series(f0)
    return fi



def get_data(locus_list, father, mother, child):
    '''根据locus_list抽取人的数据'''
    df = pd.DataFrame(columns=['locus','moGT','faGT', 'chGT','f0'])
    for record in vcf:   # 先将所有数据读进df
        locus = (record.chrom,str(record.pos))
        f0 = 1-record.info['AF'][0]
        if locus in locus_list:  # 除去f0=1的情况，不然会导致之后 arctanh(1)=inf
            mo = record.samples[mother]['GT']
            ch = record.samples[child]['GT']
            fa = record.samples[father]['GT']
            df.loc[len(df.index)] = [locus,mo,fa,ch,f0] # type: ignore
    df.set_index('locus',drop=True,inplace=True)
    return df



In [2]:
pc = 0.001
alpha = 0.001
rho1 = 0.5
rho2 = 0.1
data_path = "/home/anran/paternity/family-data/sim_seg1" 
hap_file,samples_file,vcf_file,ped_file=get_file(data_path)
persons = pd.read_table(ped_file)
df_data_init = convert_hap_samples_to_dataframe(hap_file,samples_file)

In [3]:
df_data = pd.read_csv('/home/anran/paternity/family-data/sim_data/data1.csv')
df_data.drop(['Unnamed: 0','personID'], axis=1,inplace=True)
# df_data2 = pd.read_csv('/home/anran/paternity/family-data/sim_data/data2.csv')
# df_data = pd.concat([df_data1,df_data2])
df_data.columns = df_data_init.columns

In [4]:
with open('/home/anran/paternity/family-data/sim_data/info1.json') as f:
    data = json.load(f)
start = data['start']
end = data['end']
fi_serie = get_fi(df_data,start,end)
N = fi_serie.shape[0]

linkage aera:  ('chr6', '22603023')  to  ('chr6', '22651774')


In [5]:
locus_list = fi_serie.index.to_list()
fi =  fi_serie.values
# fij = calcul_fij(df_data,fi,locus_list,N)
fi,fij = freq_pc(df_data,locus_list,fi,N,pc)

In [6]:
cmat = cMat(fi,fij,N,alpha)  
invc = invc_glassoR(cmat,rho1,rho2)
diag = ((invc-np.diag(np.diagonal(invc)))==0).all()  # 判断逆矩阵是否为对角矩阵

	 c的最小特征值 (-5.80599909613751e-15+0j)


In [7]:
emat = calcul_e(invc,fi,N) 
h = margin(fi,emat,N)
log_z = calcul_Z(emat,h,K,N)

	 emat: -2.0 0.24730314984745916
	 h: 0.18322466855360606 4.146899804423353


In [12]:
emat

array([[-2.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -2.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -1.33516545, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -2.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -2.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -2.        ]])

In [8]:
def get_fi_ind(df_data,start,end,n):
    f0={}
    totle = df_data.shape[0]
    num_col = df_data.shape[1]
    # index_available = np.append(np.arange(start),np.arange(end+1,num_col))
    # print(index_available.shape)
    index = np.random.choice(np.append(np.arange(start),np.arange(end+1,num_col)),size=n,replace=False)
    columns = df_data.columns.to_list()
    for i in index:
        locus = columns[i]
        freq = (df_data[locus]==0).sum()/totle
        f0[locus] = freq
    fi = pd.Series(f0)
    return fi

In [9]:
vcf = pysam.VariantFile(vcf_file)
df_select = get_data(locus_list,vcf,persons.father[i],persons.mother[j],persons.child[j])
log_x,log_y,c = calcul_XY(df_select)

NameError: name 'i' is not defined

In [12]:
def generate_linkage_intervals(num_linkage,num_locus,num_keys):
    '''生成给定数量的重组间隔，即随机选择一定数量的变异位点作为重组点，形成相邻两个变异位点之间的重组区间。'''
    linkage_init = random.randint(0,num_locus-num_linkage)
    keys = np.random.choice(np.arange(1,num_linkage-1), num_keys,replace=False) + linkage_init
    linkage_keys = np.sort(np.append(keys,[linkage_init,linkage_init+num_linkage]))
    return linkage_keys


In [33]:
linkage_keys = generate_linkage_intervals(8,20,2)
linkage_keys

array([ 8, 11, 14, 16])

In [38]:
temp = np.arange(20)
for j in range(20):
    if j<linkage_keys[0] or j>=linkage_keys[-1]:
        pass           
    else:
        index = np.searchsorted(linkage_keys,j,side='right') - 1
        temp[j] = temp[linkage_keys[index]]
temp

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8, 11, 11, 11, 14, 14, 16,
       17, 18, 19])

In [4]:
a = np.array([1,2])

In [7]:
type(list(map(int,a))[0])

int

In [3]:
lst=[[1,2,3],
     [4,5,6],
     [7,8,9],
     [10,11,12]]
df=pd.DataFrame(lst)
print(df)

    0   1   2
0   1   2   3
1   4   5   6
2   7   8   9
3  10  11  12


In [4]:
person_data = pd.DataFrame(columns = np.arange(df.shape[1]))
for j in range(df.shape[0]//2):
    print(j)
    person_data.loc[j] = list(zip(df.iloc[2*j].values,df.iloc[2*j+1].values))

0
1


In [5]:
person_data

,0,1,2
0,"(1, 4)","(2, 5)","(3, 6)"
1,"(7, 10)","(8, 11)","(9, 12)"


In [1]:
import random
from re import match
import pysam
import numpy as np
import pandas as pd
import os
import argparse
import json
from copy import deepcopy

from convert import convert_hap_samples_to_dataframe

def get_file(data_path):
    '''从文件夹中，得到各文件的路径'''
    for filename in os.listdir(data_path):
        if filename.endswith('.gz'):
            hap_file = os.path.join(data_path,filename)
        if filename.endswith('.sample'):
            samples_file = os.path.join(data_path,filename)
        if filename.endswith('.vcf'):
            vcf_file = os.path.join(data_path,filename)
        if filename.endswith('.ped'):
            ped_file = os.path.join(data_path,filename)
    assert (hap_file and samples_file and vcf_file and ped_file),'missing data file'
    return  hap_file,samples_file,vcf_file,ped_file

def parse_args():
    parser = argparse.ArgumentParser(description="Simulate generations of families and populations")
    parser.add_argument("dataset", help="Input data file with initial individuals")
    parser.add_argument("output_file_path", help="path for saving simulation files ")
    parser.add_argument("-g", "--num_generations", type=int, default=3, help="Number of generations to simulate (default: 3)")
    parser.add_argument("-c", "--num_couples", type=int, default=10, help="Number of initial couples (default: 10)")
    parser.add_argument("-l", "--num_linkages", type=int, default=1000, help="Number of consecutive non-recombining positions (default: 1000)")
    parser.add_argument("-p", "--num_keys", type=int, default=0, help="key points for linkage interval (default: 0)")
    parser.add_argument("-k", "--num_kids", type=int, default=2, help="Number of child for every couple (default: 2)")
    parser.add_argument("-r", "--num_recombinations", type=int, default=5, help="Number of intervals for recombination (default: 5)")

    return parser.parse_args()

def choice_person(df_data,num_couples,linkage_keys):
    start = linkage_keys[0]
    end = linkage_keys[-1]
    res0 = pd.DataFrame()
    select_people = {}
    num_people = df_data.shape[0]//2
    person_list = df_data.index.tolist()
    df_data.columns = np.arange(df_data.shape[1])
    for i in range(2*num_couples):
        index = random.randint(0,num_people)
        select_people[i] = person_list[2*index]
        select_data = deepcopy(df_data.iloc[index*2:index*2+2])
        select_data['personID'] = [i,i]
        res0 = pd.concat([res0,select_data])
    res0.set_index(np.arange(4*num_couples),drop=True,inplace=True)
    for i in range(res0.shape[0]):
        res0.iloc[i,start:end] = res0.iloc[i,start]
    return res0,select_people

def generate_linkage_intervals(num_linkage,num_locus,num_keys=0):
    linkage_init = random.randint(0,num_locus-num_linkage)
    keys = np.random.choice(np.arange(1,num_linkage-1), num_keys,replace=False) + linkage_init
    linkage_keys = np.sort(np.append(keys,[linkage_init,linkage_init+num_linkage+1]))
    return linkage_keys

def generate_recombination_intervals(num_recombinations,num_locus,linkage_keys):
    start = linkage_keys[0]
    end = linkage_keys[-1]
    num_linkage = end-start
    interval1 = range(0,start)
    interval2 = range(end+1,num_locus+1)
    points1 = start*num_recombinations//(num_locus-num_linkage)
    points2 = num_recombinations - points1
    recombination_points1 = sorted(random.sample(interval1, 2 * points1)) 
    recombination_points2 = sorted(random.sample(interval2, 2 * points2))
    recombination_points = recombination_points1 + recombination_points2
    recombination_intervals = [(recombination_points[i], recombination_points[i + 1]) for i in range(0, len(recombination_points), 2)]
    return recombination_intervals

def is_variant_in_recombination_intervals(variant_pos, recombination_intervals):
    '''检查一个变异位点是否在给定的重组区间内'''
    if recombination_intervals!=[]:
        for start, end in recombination_intervals:
            if start <= variant_pos <= end:
                return True
    return False

def sim_meosis(res,num_recombinations,linkage_keys):
    num_locus = res.shape[1]-1
    start = linkage_keys[0]
    end = linkage_keys[-1]
    temp = deepcopy(res[res.index%2==0])
    for i in range(temp.shape[0]):
        if num_recombinations>0:
            recombination_intervals = generate_recombination_intervals(num_recombinations,num_locus,linkage_keys)
        else:
            recombination_intervals = []
        for j in range(num_locus):
            if j<start or j>=end:
                if is_variant_in_recombination_intervals(j,recombination_intervals):
                    temp.iloc[i,j] = res.iloc[i+1,j]
            # else:
            #     # index = np.searchsorted(linkage_keys,j,side='right') - 1   # type: ignore
            #     temp.iloc[i,j] = res.iloc[i,start]

    return temp

def couple_legal(person1,person2,family=None):
    if not family:
        return True
    else:
        if person1 in family.keys() and person2 in family.keys():
            if family[person1] == family[person2]:
                return False
            else:
                return True
        else:
            return True

def select_parent(person_rest,family):
    persons = deepcopy(person_rest)
    parent1,parent2 = random.sample(persons,2)
    while not couple_legal(parent1,parent2,family):
        parent1,parent2 = random.sample(persons,2)
    persons.remove(parent1)
    persons.remove(parent2)
    return persons,parent1,parent2

def valid_linkage(df,linkage_keys):
    for i in range(len(linkage_keys)-1):
        start = linkage_keys[i]
        end = linkage_keys[i+1]
        a = (df.iloc[:,start]=='0').sum()
        for j in range(start,end):
            if (df.iloc[:,j]=='0').sum() != a:
                print('不满足强连锁，列数：',i)
                return False
    return True
                
def genetype(df_family,dict_family):
    family = df_family.drop(['personID'], axis=1)
    person_data = pd.DataFrame(columns = np.arange(family.shape[1]))
    print(family.shape,person_data.shape)
    for j in range(family.shape[0]//2):
        print(j)
        person_data.loc[j] = list(zip(family.iloc[2*j].values,family.iloc[2*j+1].values))

    person_data.to_csv('/home/anran/paternity/family-data/sim_data/person.csv',index=False)

    person = []
    for key in dict_family.keys():
        if key != 'linkage_keys':
            person.append([int(key)]+list(map(int,family[key])))
    persons = pd.DataFrame(person,columns = ['child','father','mother'])
    persons.to_csv('/home/anran/paternity/family-data/sim_data/ped.csv',index=False)
    return 0

def dataset(df_data,linkage_keys,args):
    num_generation = args.num_generations
    num_couple = args.num_couples
    num_child = args.num_kids
    num_recombination = args.num_recombinations
    result_df = pd.DataFrame()

    res0,family = choice_person(df_data,num_couple,linkage_keys)
    assert valid_linkage(res0,linkage_keys),'dataset select data non linkage'
    result_df = pd.concat([result_df,res0])
    personID = 2*num_couple
    for g in range(num_generation):
        print('#####################',g)
        couple = []
        for k in range(num_child):
            print('#########',k)
            temp = sim_meosis(res0,num_recombination,linkage_keys)
            persons_rest = list(map(int,list(temp['personID'].values)))
            # print(temp.shape,persons_rest)
            if k==0:
                while persons_rest!=[]:
                    persons_rest,parent1,parent2 = select_parent(persons_rest,family)
                    # print('-----------------',parent1,parent2,persons_rest)
                    couple.append((parent1,parent2))
                    family[personID] = [parent1,parent2]
                    res_temp = deepcopy(temp[temp['personID'].isin([parent1,parent2])])
                    res_temp['personID'] = [personID,personID]
                    result_df = pd.concat([result_df,res_temp])
                    personID += 1
            else:
                for (parent1,parent2) in couple:
                    family[personID] = [parent1,parent2]
                    res_temp = deepcopy(temp[temp['personID'].isin([parent1,parent2])])
                    res_temp['personID'] = [personID,personID]
                    result_df = pd.concat([result_df,res_temp])
                    personID += 1
        res0 = result_df.iloc[2*personID-4*num_couple:2*personID]
        res0.set_index(np.arange(res0.shape[0]),drop=True,inplace=True)

    assert valid_linkage(result_df,linkage_keys),'dataset final data non linkage'
    family['linkage_keys'] = list(map(int,linkage_keys))
    result_df.to_csv(args.output_file_path + 'df_data.csv',index=False)
    with open(args.output_file_path+"df_data.json","w",encoding='utf-8') as f:
        json.dump(family,f)
    return 0

def test_dataset(df_data,linkage_keys,args):
    num_couple = 5
    num_recombination = args.num_recombinations
    result_df = pd.DataFrame()
    family = {}

    res0,_ = choice_person(df_data,num_couple,linkage_keys)
    assert valid_linkage(res0,linkage_keys),'family select data non linkage'
    result_df = pd.concat([result_df,res0])
    personID = 2*num_couple
   
    temp = sim_meosis(res0,num_recombination,linkage_keys)
    persons_rest = list(map(int,list(temp['personID'].values)))
    # print(temp.shape,persons_rest)

    while persons_rest!=[]:
        persons_rest,parent1,parent2 = select_parent(persons_rest,family)
        print('-----------------',parent1,parent2,persons_rest)
        family[personID] = [parent1,parent2]
        res_temp = deepcopy(temp[temp['personID'].isin([parent1,parent2])])
        res_temp['personID'] = [personID,personID]
        result_df = pd.concat([result_df,res_temp])
        personID += 1

    assert valid_linkage(result_df,linkage_keys),'family final data non linkage'
    # result_df.to_csv(args.output_file_path + 'family.csv',index=False)
    with open(args.output_file_path+"family.json","w",encoding='utf-8') as f:
        json.dump(family,f)
    genetype(result_df,family)
    return 0